In [2]:
import sys,os
folder_name="march_madness"
BASE_DIR=os.path.abspath(".").split(folder_name)[0]+folder_name
DATA_ROOT=os.path.join(BASE_DIR,"data")
#todo setup shared utilities folder
sys.path.insert(0, DATA_ROOT)

In [3]:
import pandas as pd
pd.options.display.max_rows=5#So we don't have to always write head
pd.options.display.max_columns=999
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import featuretools as ft

In [4]:
regular_season_results_df=pd.read_csv(DATA_ROOT+"/MRegularSeasonDetailedResults.csv")
regular_season_results_df['game_type']='regular_season'
tourney_results=pd.read_csv(DATA_ROOT+"/MNCAATourneyDetailedResults.csv")
tourney_results['game_type']='march_madness'
season_results=pd.concat([regular_season_results_df, tourney_results]).reset_index(drop=True)
season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,game_type
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,regular_season
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,regular_season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97800,2019,152,1438,63,1120,62,N,0,25,51,7,19,6,12,5,26,15,8,1,9,12,21,55,9,31,11,14,9,24,9,5,3,3,12,march_madness
97801,2019,154,1438,85,1403,77,N,1,27,59,11,24,20,23,11,28,15,11,4,3,15,27,63,10,30,13,15,9,23,9,8,6,3,18,march_madness


In [5]:
seasons=pd.read_csv(DATA_ROOT+"/MSeasons.csv").set_index("Season",drop=False)
seasons['DayZero']=pd.to_datetime(seasons['DayZero'])
seasons

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
Season,,,,,,
1985,1985,1984-10-29,East,West,Midwest,Southeast
1986,1986,1985-10-28,East,Midwest,Southeast,West
...,...,...,...,...,...,...
2020,2020,2019-11-04,NA1,NA2,NA3,NA4
2021,2021,2020-11-02,East,West,Midwest,South


In [6]:
teams=pd.read_csv(DATA_ROOT+"/MTeams.csv")
team_conferences=pd.read_csv(DATA_ROOT+"/MTeamConferences.csv")
tourney_seeds=pd.read_csv(DATA_ROOT+"/MNCAATourneySeeds.csv")
tourney_seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
...,...,...,...
2352,2021,Z15,1331
2353,2021,Z16,1216


In [7]:
from itertools import combinations
possible_matchups=[]
for c in combinations(sorted(teams['TeamID']),2):
    possible_matchups.append(c)
possible_matchups_df=pd.DataFrame(possible_matchups,columns=["team_1_id","team_2_id"]).astype(str)
possible_matchups_df['matchup_id']=possible_matchups_df['team_1_id']+"_"+possible_matchups_df['team_2_id']
possible_matchups_df

,team_1_id,team_2_id,matchup_id
0,1101,1102,1101_1102
1,1101,1103,1101_1103
...,...,...,...
68633,1469,1471,1469_1471
68634,1470,1471,1470_1471


In [7]:
season_results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,game_type
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20,regular_season
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16,regular_season
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97800,2019,152,1438,63,1120,62,N,0,25,51,7,19,6,12,5,26,15,8,1,9,12,21,55,9,31,11,14,9,24,9,5,3,3,12,march_madness
97801,2019,154,1438,85,1403,77,N,1,27,59,11,24,20,23,11,28,15,11,4,3,15,27,63,10,30,13,15,9,23,9,8,6,3,18,march_madness


In [8]:
#prep regular season game_log same as tourney games
import datetime
def get_game_index(row):
    team_1_id,team_2_id=sorted([row['WTeamID'],row['LTeamID']])
    return "%s_%s_%s_%s"%(row['Season'],row['DayNum'],team_1_id,team_2_id)
def get_season_game_index(row):
    team_1_id,team_2_id=sorted([row['WTeamID'],row['LTeamID']])
    return "%s_%s_%s"%(row['Season'],team_1_id,team_2_id)
def get_game_daytime(row):
    #not an acutal "day". This uses Season and DayNum without a first day to index the day_num
    
    game_day=seasons.loc[row['Season']]['DayZero']+datetime.timedelta(days=row['DayNum'])
    return game_day
    
season_results['game_index']=season_results.apply(get_game_index, axis=1)
season_results['season_game_index']=season_results.apply(get_game_index, axis=1)
season_results['game_datetime']=pd.to_datetime(season_results.apply(get_game_daytime, axis=1))

def get_team_game_log(results_df,winner=True):
    team_game_log=season_results.copy()
    prefixes=["team_","opp_"]
    if not winner:
        prefixes=["opp_","team_"]
    rename_dict={}
    for col in season_results.columns:
        if col=="WLoc":
            continue
        elif col.startswith("W"):
            rename_dict[col]=prefixes[0]+col[1:]
        elif col.startswith("L") and col not in ["Loc"]:
            rename_dict[col]=prefixes[1]+col[1:]
    team_game_log=team_game_log.rename(columns=rename_dict)
    return team_game_log
winner_games=get_team_game_log(season_results, winner=True)
loser_games=get_team_game_log(season_results, winner=False)
team_game_log=pd.concat([winner_games,loser_games],sort=False)
team_game_log=team_game_log.sort_values(by=["Season","DayNum","game_index"])
team_game_log['team_margin']=team_game_log['team_Score']-team_game_log['opp_Score']
team_game_log['opp_margin']=-team_game_log['team_margin']
team_game_log['team_wins']=team_game_log['team_margin']>0
team_game_log['opp_wins']=~team_game_log['team_wins']
#teams can't tie in basketball, above code assumes that
assert(len(team_game_log[team_game_log['team_margin']==0])==0)
def get_game_loc(row):
    if row['team_wins']:
        return row['WLoc']
    else:
        if row['WLoc']=='H':
            return 'A'
        elif row['WLoc']=='A':
            return 'H'
        else:
            return row['WLoc']
def get_matchup_id(row):
    ids=sorted([row['team_TeamID'], row['opp_TeamID']])
    return "%s_%s"%tuple(ids)
team_game_log['team_game_loc']=team_game_log.apply(get_game_loc,axis=1)
del team_game_log['WLoc']
team_game_log['matchup_id']=team_game_log.apply(get_matchup_id , axis=1)
team_game_log['team_season_id']=team_game_log['Season'].astype(str)+"_"+team_game_log['team_TeamID'].astype(str)
team_game_log['opp_season_id']=team_game_log['Season'].astype(str)+"_"+team_game_log['opp_TeamID'].astype(str)
team_game_log['team_game_index']=team_game_log['game_index']+"_"+team_game_log['team_TeamID'].astype(str)
team_game_log['opp_game_index']=team_game_log['game_index']+"_"+team_game_log['team_TeamID'].astype(str)

In [9]:
#team_1 should always be the same team_1 with respect to a matchup_id
matchups=team_game_log.sort_values(by="team_game_index").groupby("game_index",as_index=False).first()
rename_dict={}
for c in matchups.columns:
    rename_dict[c]=c.replace("team_","team_1_").replace("opp_","team_2_")
matchups=matchups.rename(columns=rename_dict)
matchups=matchups[sorted(matchups.columns)].sort_values(by=["game_datetime"])
matchups

,DayNum,NumOT,Season,game_datetime,game_index,game_type,matchup_id,season_game_index,team_1_Ast,team_1_Blk,team_1_DR,team_1_FGA,team_1_FGA3,team_1_FGM,team_1_FGM3,team_1_FTA,team_1_FTM,team_1_OR,team_1_PF,team_1_Score,team_1_Stl,team_1_TO,team_1_TeamID,team_1_game_index,team_1_game_loc,team_1_margin,team_1_season_id,team_1_wins,team_2_Ast,team_2_Blk,team_2_DR,team_2_FGA,team_2_FGA3,team_2_FGM,team_2_FGM3,team_2_FTA,team_2_FTM,team_2_OR,team_2_PF,team_2_Score,team_2_Stl,team_2_TO,team_2_TeamID,team_2_game_index,team_2_margin,team_2_season_id,team_2_wins
417,10,0,2003,2002-11-14,2003_10_1272_1393,regular_season,1272_1393,2003_10_1272_1393,16,4,28,62,20,26,8,19,10,15,18,70,4,13,1272,2003_10_1272_1393_1272,N,7,2003_1272,True,7,6,25,67,24,24,6,20,9,20,16,63,8,12,1393,2003_10_1272_1393_1272,-7,2003_1393,False
416,10,0,2003,2002-11-14,2003_10_1104_1328,regular_season,1104_1328,2003_10_1104_1328,13,1,24,58,14,27,3,18,11,14,22,68,7,23,1104,2003_10_1104_1328_1104,N,6,2003_1104,True,8,2,22,53,10,22,2,22,16,10,20,62,9,18,1328,2003_10_1104_1328_1104,-6,2003_1328,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95186,132,1,2021,2021-03-14,2021_132_1228_1326,regular_season,1228_1326,2021_132_1228_1326,15,2,27,64,21,29,9,32,24,13,23,91,5,10,1228,2021_132_1228_1326_1228,N,3,2021_1228,True,11,4,27,65,25,29,8,30,22,8,26,88,7,8,1326,2021_132_1228_1326_1228,-3,2021_1326,False
95183,132,0,2021,2021-03-14,2021_132_1104_1261,regular_season,1104_1261,2021_132_1104_1261,16,8,31,76,36,33,12,5,2,16,15,80,4,9,1104,2021_132_1104_1261_1104,N,1,2021_1104,True,10,5,26,74,27,30,10,13,9,15,13,79,6,5,1261,2021_132_1104_1261_1104,-1,2021_1261,False


In [10]:
matchups.columns

Index(['DayNum', 'NumOT', 'Season', 'game_datetime', 'game_index', 'game_type',
       'matchup_id', 'season_game_index', 'team_1_Ast', 'team_1_Blk',
       'team_1_DR', 'team_1_FGA', 'team_1_FGA3', 'team_1_FGM', 'team_1_FGM3',
       'team_1_FTA', 'team_1_FTM', 'team_1_OR', 'team_1_PF', 'team_1_Score',
       'team_1_Stl', 'team_1_TO', 'team_1_TeamID', 'team_1_game_index',
       'team_1_game_loc', 'team_1_margin', 'team_1_season_id', 'team_1_wins',
       'team_2_Ast', 'team_2_Blk', 'team_2_DR', 'team_2_FGA', 'team_2_FGA3',
       'team_2_FGM', 'team_2_FGM3', 'team_2_FTA', 'team_2_FTM', 'team_2_OR',
       'team_2_PF', 'team_2_Score', 'team_2_Stl', 'team_2_TO', 'team_2_TeamID',
       'team_2_game_index', 'team_2_margin', 'team_2_season_id',
       'team_2_wins'],
      dtype='object')

In [11]:
#get rid of duplicate information here
matchups=matchups[
    ['game_datetime', 'game_index', 'game_type',
       'matchup_id','team_1_season_id','team_2_season_id',
    'team_1_Ast', 'team_1_Blk', 'team_1_DR', 'team_1_FGA',
       'team_1_FGA3', 'team_1_FGM', 'team_1_FGM3', 'team_1_FTA', 'team_1_FTM',
       'team_1_OR', 'team_1_PF', 'team_1_Score', 'team_1_Stl', 'team_1_TO',
       'team_1_TeamID',  'team_1_margin','team_1_game_loc',
        'team_1_wins', 'team_2_Ast', 'team_2_Blk',
       'team_2_DR', 'team_2_FGA', 'team_2_FGA3', 'team_2_FGM', 'team_2_FGM3',
       'team_2_FTA', 'team_2_FTM', 'team_2_OR', 'team_2_PF', 'team_2_Score',
       'team_2_Stl', 'team_2_TO', 'team_2_TeamID', 
       ]
]

In [12]:
matchups

,game_datetime,game_index,game_type,matchup_id,team_1_season_id,team_2_season_id,team_1_Ast,team_1_Blk,team_1_DR,team_1_FGA,team_1_FGA3,team_1_FGM,team_1_FGM3,team_1_FTA,team_1_FTM,team_1_OR,team_1_PF,team_1_Score,team_1_Stl,team_1_TO,team_1_TeamID,team_1_margin,team_1_game_loc,team_1_wins,team_2_Ast,team_2_Blk,team_2_DR,team_2_FGA,team_2_FGA3,team_2_FGM,team_2_FGM3,team_2_FTA,team_2_FTM,team_2_OR,team_2_PF,team_2_Score,team_2_Stl,team_2_TO,team_2_TeamID
417,2002-11-14,2003_10_1272_1393,regular_season,1272_1393,2003_1272,2003_1393,16,4,28,62,20,26,8,19,10,15,18,70,4,13,1272,7,N,True,7,6,25,67,24,24,6,20,9,20,16,63,8,12,1393
416,2002-11-14,2003_10_1104_1328,regular_season,1104_1328,2003_1104,2003_1328,13,1,24,58,14,27,3,18,11,14,22,68,7,23,1104,6,N,True,8,2,22,53,10,22,2,22,16,10,20,62,9,18,1328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95186,2021-03-14,2021_132_1228_1326,regular_season,1228_1326,2021_1228,2021_1326,15,2,27,64,21,29,9,32,24,13,23,91,5,10,1228,3,N,True,11,4,27,65,25,29,8,30,22,8,26,88,7,8,1326
95183,2021-03-14,2021_132_1104_1261,regular_season,1104_1261,2021_1104,2021_1261,16,8,31,76,36,33,12,5,2,16,15,80,4,9,1104,1,N,True,10,5,26,74,27,30,10,13,9,15,13,79,6,5,1261


In [13]:
matchups['season']=matchups['team_1_season_id'].apply(lambda x: int(x.split("_")[0]))

In [14]:
matchups['game_end']=matchups['game_datetime']+pd.Timedelta("1 day")
matchups

,game_datetime,game_index,game_type,matchup_id,team_1_season_id,team_2_season_id,team_1_Ast,team_1_Blk,team_1_DR,team_1_FGA,team_1_FGA3,team_1_FGM,team_1_FGM3,team_1_FTA,team_1_FTM,team_1_OR,team_1_PF,team_1_Score,team_1_Stl,team_1_TO,team_1_TeamID,team_1_margin,team_1_game_loc,team_1_wins,team_2_Ast,team_2_Blk,team_2_DR,team_2_FGA,team_2_FGA3,team_2_FGM,team_2_FGM3,team_2_FTA,team_2_FTM,team_2_OR,team_2_PF,team_2_Score,team_2_Stl,team_2_TO,team_2_TeamID,season,game_end
417,2002-11-14,2003_10_1272_1393,regular_season,1272_1393,2003_1272,2003_1393,16,4,28,62,20,26,8,19,10,15,18,70,4,13,1272,7,N,True,7,6,25,67,24,24,6,20,9,20,16,63,8,12,1393,2003,2002-11-15
416,2002-11-14,2003_10_1104_1328,regular_season,1104_1328,2003_1104,2003_1328,13,1,24,58,14,27,3,18,11,14,22,68,7,23,1104,6,N,True,8,2,22,53,10,22,2,22,16,10,20,62,9,18,1328,2003,2002-11-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95186,2021-03-14,2021_132_1228_1326,regular_season,1228_1326,2021_1228,2021_1326,15,2,27,64,21,29,9,32,24,13,23,91,5,10,1228,3,N,True,11,4,27,65,25,29,8,30,22,8,26,88,7,8,1326,2021,2021-03-15
95183,2021-03-14,2021_132_1104_1261,regular_season,1104_1261,2021_1104,2021_1261,16,8,31,76,36,33,12,5,2,16,15,80,4,9,1104,1,N,True,10,5,26,74,27,30,10,13,9,15,13,79,6,5,1261,2021,2021-03-15


In [15]:
matchups.columns

Index(['game_datetime', 'game_index', 'game_type', 'matchup_id',
       'team_1_season_id', 'team_2_season_id', 'team_1_Ast', 'team_1_Blk',
       'team_1_DR', 'team_1_FGA', 'team_1_FGA3', 'team_1_FGM', 'team_1_FGM3',
       'team_1_FTA', 'team_1_FTM', 'team_1_OR', 'team_1_PF', 'team_1_Score',
       'team_1_Stl', 'team_1_TO', 'team_1_TeamID', 'team_1_margin',
       'team_1_game_loc', 'team_1_wins', 'team_2_Ast', 'team_2_Blk',
       'team_2_DR', 'team_2_FGA', 'team_2_FGA3', 'team_2_FGM', 'team_2_FGM3',
       'team_2_FTA', 'team_2_FTM', 'team_2_OR', 'team_2_PF', 'team_2_Score',
       'team_2_Stl', 'team_2_TO', 'team_2_TeamID', 'season', 'game_end'],
      dtype='object')

In [19]:
possible_matchups[0]

(1101, 1102)

In [16]:
#del matchups_es

In [16]:
from woodwork.logical_types import Categorical

matchups_es = ft.EntitySet(id="matchups_es")
matchups_es = matchups_es.add_dataframe(dataframe_name="possible_matchups",
                             dataframe=possible_matchups_df,
                             index="matchup_id"
                             )
matchups_es = matchups_es.add_dataframe(dataframe_name="matchup_logs",
                            dataframe=matchups,
                            index="game_index",
                            time_index="game_datetime",
                            logical_types={
                                "team_1_TeamID": Categorical,
                                "team_2_TeamID": Categorical
                            },
                            secondary_time_index={
                                "game_end": 
                                  [ 'team_1_Ast', 'team_1_Blk', 'team_1_DR', 'team_1_FGA',
                                   'team_1_FGA3', 'team_1_FGM', 'team_1_FGM3', 'team_1_FTA', 'team_1_FTM',
                                   'team_1_OR', 'team_1_PF', 'team_1_Score', 'team_1_Stl', 'team_1_TO',
                                   'team_1_TeamID',  'team_1_margin',
                                    'team_1_wins', 'team_2_Ast', 'team_2_Blk',
                                   'team_2_DR', 'team_2_FGA', 'team_2_FGA3', 'team_2_FGM', 'team_2_FGM3',
                                   'team_2_FTA', 'team_2_FTM', 'team_2_OR', 'team_2_PF', 'team_2_Score',
                                   'team_2_Stl', 'team_2_TO', 'team_2_TeamID', 
                                    ]}
                             )
matchups_es.add_relationship(
    "possible_matchups",
    'matchup_id',
    'matchup_logs',
    'matchup_id'
)

#matchups_es.plot()

Entityset: matchups_es
  DataFrames:
    possible_matchups [Rows: 68635, Columns: 3]
    matchup_logs [Rows: 97802, Columns: 41]
  Relationships:
    matchup_logs.matchup_id -> possible_matchups.matchup_id

In [19]:
matchups_es["matchup_logs"]["team_1_wins"].interesting_values = [False, True]

matchups_es["matchup_logs"]["game_type"].interesting_values = ["regular_season", "march_madness"]

In [20]:
matchup_logs=matchups_es['matchup_logs']
tourney_starts=matchup_logs[matchup_logs['game_type']=='march_madness'].groupby("season").first()['game_datetime']
tourney_starts

season
2003   2003-03-18
2004   2004-03-16
          ...    
2018   2018-03-13
2019   2019-03-19
Name: game_datetime, Length: 17, dtype: datetime64[ns]

In [21]:
def make_tourney_labels():
    matchup_logs = matchups_es["matchup_logs"]

    time_variable="game_end"
    full_training_data = pd.DataFrame()
    valid_prediction_data = pd.DataFrame()
    for season, tourney_start in tourney_starts.iteritems():
        tourney_data=matchup_logs[(matchup_logs["game_end"] <= tourney_start) & (matchup_logs["season"]==season)]
        full_training_data = pd.concat((full_training_data, tourney_data))
        valid_pred_data = tourney_data[tourney_data['matchup_id'].isin(
            full_training_data['matchup_id'])]
        valid_prediction_data = pd.concat(
            (valid_prediction_data, valid_pred_data))
        extra_cols=['team_1_wins']#, 'team_2_wins',"matchup_id"]
    
    labels = valid_prediction_data[["game_index","game_datetime",]+extra_cols]
    labels["time"] = labels['game_datetime']
    labels = labels[["game_index", "time"]+extra_cols]

    return labels.reset_index(drop=True)
def make_day_of_labels(es, training_window, cutoff_time,
                prediction_window):

    prediction_window_end = cutoff_time + prediction_window
    t_start = cutoff_time - training_window

    matchup_logs = es["matchup_logs"]
    
    time_index="game_datetime"
    training_data = matchup_logs[(matchup_logs[time_index] <= cutoff_time) & (matchup_logs[time_index] > t_start)]
    prediction_data = matchup_logs[(matchup_logs[time_index] > cutoff_time) & (matchup_logs[time_index] < prediction_window_end)]
    
    matchup_ids_in_training = training_data['matchup_id'].unique()
    valid_pred_data = prediction_data[prediction_data['matchup_id'].isin(matchup_ids_in_training)]
    extra_cols=['team_1_wins']
    labels = valid_pred_data[["game_index","game_datetime",]+extra_cols]
    labels["time"] = labels['game_datetime']
    del labels['game_datetime']
    labels = labels[["game_index", "time",]+extra_cols]

    return labels.reset_index(drop=True)

In [22]:

tourney_labels=make_tourney_labels()
#tourney_labels[tourney_labels['game_index']=='2016_145_1242_1437']
tourney_labels

,game_index,time,team_1_wins
0,2003_10_1104_1328,2002-11-14,True
1,2003_10_1272_1393,2002-11-14,True
...,...,...,...
87502,2019_132_1276_1277,2019-03-17,False
87503,2019_132_1382_1387,2019-03-17,False


In [23]:
day_of_labels=make_day_of_labels(matchups_es, 
    cutoff_time = pd.Timestamp('February 3, 2003'),
                                prediction_window = ft.Timedelta("5 weeks"),
                                training_window = ft.Timedelta("12 weeks")
           )

In [24]:
pd.options.display.max_rows=6
matchup_logs[matchup_logs['matchup_id']=='1242_1437']

,game_datetime,game_index,game_type,matchup_id,team_1_season_id,team_2_season_id,team_1_Ast,team_1_Blk,team_1_DR,team_1_FGA,team_1_FGA3,team_1_FGM,team_1_FGM3,team_1_FTA,team_1_FTM,team_1_OR,team_1_PF,team_1_Score,team_1_Stl,team_1_TO,team_1_TeamID,team_1_margin,team_1_game_loc,team_1_wins,team_2_Ast,team_2_Blk,team_2_DR,team_2_FGA,team_2_FGA3,team_2_FGM,team_2_FGM3,team_2_FTA,team_2_FTM,team_2_OR,team_2_PF,team_2_Score,team_2_Stl,team_2_TO,team_2_TeamID,season,game_end
2004_60_1242_1437,2004-01-02,2004_60_1242_1437,regular_season,1242_1437,2004_1242,2004_1437,17,7,27,51,11,23,5,41,35,8,24,86,6,15,1242,7,H,True,19,3,23,67,35,25,13,19,16,18,32,79,8,18,1437,2004,2004-01-03
2005_82_1242_1437,2005-01-22,2005_82_1242_1437,regular_season,1242_1437,2005_1242,2005_1437,15,7,18,55,17,24,7,12,7,14,14,62,5,22,1242,-21,A,False,10,6,18,55,19,28,12,16,15,10,15,83,15,15,1437,2005,2005-01-23
2008_144_1242_1437,2008-03-28,2008_144_1242_1437,march_madness,1242_1437,2008_1242,2008_1437,16,5,26,45,19,24,8,20,16,7,18,72,8,18,1242,15,N,True,5,2,15,59,17,21,3,18,12,15,17,57,10,14,1437,2008,2008-03-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018_152_1242_1437,2018-03-31,2018_152_1242_1437,march_madness,1242_1437,2018_1242,2018_1437,8,1,23,62,21,28,7,20,16,10,12,79,5,9,1242,-16,N,False,20,5,26,65,40,36,18,7,5,7,14,95,3,10,1437,2018,2018-04-01
2019_40_1242_1437,2018-12-15,2019_40_1242_1437,regular_season,1242_1437,2019_1242,2019_1437,9,2,26,46,15,23,5,30,23,9,19,74,4,14,1242,3,H,True,8,2,16,59,28,24,8,19,15,10,21,71,5,9,1437,2019,2018-12-16
2020_47_1242_1437,2019-12-21,2020_47_1242_1437,regular_season,1242_1437,2020_1242,2020_1437,14,5,31,55,13,24,3,10,4,10,12,55,6,12,1242,-1,A,False,14,0,26,63,41,22,10,2,2,10,14,56,6,9,1437,2020,2019-12-22


In [25]:
#tourney_labels=tourney_labels[tourney_labels['game_index']=='2016_145_1242_1437']
tourney_labels

,game_index,time,team_1_wins
0,2003_10_1104_1328,2002-11-14,True
1,2003_10_1272_1393,2002-11-14,True
2,2003_11_1186_1458,2002-11-15,False
...,...,...,...
87501,2019_132_1217_1463,2019-03-17,False
87502,2019_132_1276_1277,2019-03-17,False
87503,2019_132_1382_1387,2019-03-17,False


In [26]:
matchup_logs.loc[["2016_145_1242_1437"]]

,game_datetime,game_index,game_type,matchup_id,team_1_season_id,team_2_season_id,team_1_Ast,team_1_Blk,team_1_DR,team_1_FGA,team_1_FGA3,team_1_FGM,team_1_FGM3,team_1_FTA,team_1_FTM,team_1_OR,team_1_PF,team_1_Score,team_1_Stl,team_1_TO,team_1_TeamID,team_1_margin,team_1_game_loc,team_1_wins,team_2_Ast,team_2_Blk,team_2_DR,team_2_FGA,team_2_FGA3,team_2_FGM,team_2_FGM3,team_2_FTA,team_2_FTM,team_2_OR,team_2_PF,team_2_Score,team_2_Stl,team_2_TO,team_2_TeamID,season,game_end
2016_145_1242_1437,2016-03-26,2016_145_1242_1437,march_madness,1242_1437,2016_1242,2016_1437,11,2,26,50,22,23,6,11,7,6,19,59,5,16,1242,-5,N,False,7,2,23,52,18,21,4,19,18,5,16,64,11,9,1437,2016,2016-03-27


In [27]:
matchup_logs = matchups_es['matchup_logs']
matchup_logs['last_time_index']=matchup_logs['game_end']
matchup_logs.columns

Index(['game_datetime', 'game_index', 'game_type', 'matchup_id',
       'team_1_season_id', 'team_2_season_id', 'team_1_Ast', 'team_1_Blk',
       'team_1_DR', 'team_1_FGA', 'team_1_FGA3', 'team_1_FGM', 'team_1_FGM3',
       'team_1_FTA', 'team_1_FTM', 'team_1_OR', 'team_1_PF', 'team_1_Score',
       'team_1_Stl', 'team_1_TO', 'team_1_TeamID', 'team_1_margin',
       'team_1_game_loc', 'team_1_wins', 'team_2_Ast', 'team_2_Blk',
       'team_2_DR', 'team_2_FGA', 'team_2_FGA3', 'team_2_FGM', 'team_2_FGM3',
       'team_2_FTA', 'team_2_FTM', 'team_2_OR', 'team_2_PF', 'team_2_Score',
       'team_2_Stl', 'team_2_TO', 'team_2_TeamID', 'season', 'game_end',
       'last_time_index'],
      dtype='object')

In [28]:
labels=tourney_labels
feature_matrix, features = ft.dfs(target_dataframe_name="matchup_logs", 
                                  cutoff_time=labels,
                                  training_window=ft.Timedelta("16 years"),
                                  entityset=matchups_es,
                                  cutoff_time_in_index=True
                                 )
# encode categorical values
fm_encoded, features_encoded = ft.encode_features(feature_matrix,
                                                  features)

print("Number of features %s" % len(features_encoded))
fm_encoded

KeyError: 'last_time_index'

In [ ]:
matchup_logs[matchup_logs['matchup_id']=='1242_1437'].iloc[:5].mean()['team_1_Score']

In [ ]:
fm_encoded.loc[["2016_145_1242_1437"]]

# Machine Learning

In [ ]:
X = fm_encoded.reset_index().merge(labels, on=["game_index","time"])

X.drop(["game_index", "time"], axis=1, inplace=True)
X = X.fillna(0)
y = X.pop("team_1_wins")
X

In [ ]:
X['possible_matchups.MODE(matchup_logs.team_1_game_loc) = N'].value_counts()

In [29]:
from sklearn.ensemble import *
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                         scoring="roc_auc", verbose=True)

"AUC %.2f +/- %.2f" % (scores.mean(), scores.std())

NameError: name 'X' is not defined

In [ ]:
from sklearn.model_selection import train_test_split
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1, max_depth=2)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=23)

clf.fit(X_train, y_train, )
import utils
from importlib import reload
reload(utils)
top_features = utils.feature_importances(clf, X.columns, n=20)

y_hat_train=clf.predict_proba(X_train)[:,1]
y_hat_test=clf.predict_proba(X_test)[:,1]
print()
print("Log Loss")
print("TRAIN:",log_loss(y_train,y_hat_train))
print("TEST:",log_loss(y_test,y_hat_test))
print()
print("Accuracy")
print("TRAIN:",accuracy_score(y_train, clf.predict(X_train)))
print("TEST:",accuracy_score(y_test, clf.predict(X_test)))

In [ ]:
def game_index_to_kaggle_id(game_index):
    vals=game_index.split("_")
    vals.pop(1)
    return "_".join(vals)
labels['ID']=labels['game_index'].apply(game_index_to_kaggle_id)
labels['Pred']=clf.predict_proba(X)[:,1]
labels

In [ ]:
import datetime
def get_season_game_index(row):
    team_1_id,team_2_id=sorted([row['Team1_ID'],row['Team2_ID']])
    return "%s_%s_%s"%(row['Season'],team_1_id,team_2_id)
possible_games_df=pd.DataFrame()
for season, seeds in tourney_seeds.groupby("Season"):
    possible_games=list(combinations(sorted(tourney_seeds[tourney_seeds['Season']==season]['TeamID']),2))
    df=pd.DataFrame(possible_games,columns=["Team1_ID","Team2_ID"])
    df['Season']=season
    possible_games_df=possible_games_df.append(df)
possible_games_df=possible_games_df[possible_games_df['Season']>=2014]

possible_games_df['TeamID']=possible_games_df.apply(lambda row: min([row["Team1_ID"],row["Team2_ID"]]),axis=1)
possible_games_df['ID']=possible_games_df.apply(lambda row: "%s_%s_%s"%(row['Season'],min([row["Team1_ID"],row["Team2_ID"]]),max([row["Team1_ID"],row["Team2_ID"]])), axis=1)
possible_games_df

In [ ]:
predictions=possible_games_df.merge(labels,on=["ID"], how="left")
predictions['Pred']=predictions['Pred'].fillna(.5)

In [ ]:
predictions[['ID','Pred']].to_csv("matchups_model.csv",index=False)

In [ ]:
predictions[['ID','Pred']]